In [1]:
from elastic_db.elasticsearch import elastic_conection, NLPmodelIndex

es = elastic_conection()
index = NLPmodelIndex(es=es, workspace_id="dc1e7b3d-9137-4a20-a99c-d0d2029ef170")
index.index_name

'nlp_model-dc1e7b3d-9137-4a20-a99c-d0d2029ef170-03012021112419'

# Function to talk with Omni NLU model

In [2]:
from tensorflow.keras.models import load_model
import pandas as pd
import pickle
import json

with open("data/intents_dictionary_tfidf.json", 'r') as f:
      intents_dictionary = json.load(f)

omni_model = load_model('nlp_models/bestModel-62.98acc-2020-01-27.h5')
tf_idf_corpus = pickle.load(open("nlp_models/tf_idf_corpus.pickle", "rb"))
df_test = pd.read_csv("data/test_data.csv", sep="|")

In [3]:
import numpy as np

query = {
  "tokenizer" : "classic",
  "filter" : [
              "lowercase",
              "asciifolding",
              {"type": "stop", "stopwords": "_portuguese_"},
              {"type": "stemmer", "language": "brazilian"}],
  "text" : ""
}

def get_sentece_features(sentence):
    query["text"] = sentence
    result = es.indices.analyze(index=index.index_name, body=query)
    new_sentence = " ".join([token["token"] for token in result["tokens"]])
    sentence_features = tf_idf_corpus.transform([new_sentence]).toarray()
    return sentence_features 

def omni_predict(sentence):
    sentence_features = get_sentece_features(sentence)
    result = omni_model.predict(sentence_features)
    idx = np.argmax(result)
    intent, score = intents_dictionary[str(idx)], result[0][idx]
    return intent, score

In [5]:
sentence = "Adotei recentemente meu filho e preciso colocar ele no meu plano odontologico"
omni_predict(sentence)

('Incluir_Colaborador_Convenio', 0.67299044)

# Function to talk with Watson Model

In [7]:
import os

API_KEY = "nA36HqlqQkQSDf0EywIeYGPl0qI_nNvLqvBUI_zhhUmR" # os.environ['API_KEY']
SERVICE_URL = "https://api.us-south.assistant.watson.cloud.ibm.com/instances/12c017ac-1bb4-4364-8493-9cc9ba98aa26" # os.environ['SERVICE_URL']
ASSISTANT_ID = "11cdca6b-e73e-48fc-8523-13d9ee68df8b" # os.environ['ASSISTANT_ID']

In [8]:
import json
from ibm_watson import AssistantV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

authenticator = IAMAuthenticator(API_KEY)
assistant = AssistantV1(
    version='2020-04-01',
    authenticator = authenticator
)

assistant.set_service_url(SERVICE_URL)

def watson_predict(sentence):
    response = assistant.message(
        workspace_id=ASSISTANT_ID,
        input={
            'message_type': 'text',
            'text': sentence
        }
    ).get_result()
    if response.get("intents") and response["intents"]:
        return (response["intents"][0]["intent"], response["intents"][0]["confidence"])
    else:
        return "failed to identify", -1.0

In [9]:
watson_predict(sentence)

('Ajuste_Informacao_Convenio', 0.3724282860755921)

# Battle Omni NLP API vs. Watson Assistent 

In [10]:
omni_right_answer = 0
watson_right_answer = 0
for _, row in df_test.iterrows():
    omni_result = omni_predict(row.example)
    watson_result = watson_predict(row.example)
    
    print("-------------------------------------------")
    print("sentence: %s"%row.example)
    print("\n")
    print("Omni NLP API\n")
    print("predict_intent: %s"%omni_result[0])
    print("score: %.2f"%omni_result[1])
    print("right_intent: %s"%row.intent)
    if omni_result[0] == row.intent:
        omni_right_answer += 1
        print("is it right?: yes")
    else:
         print("is it right?: no")
    print("quantity of right answers: %s" %omni_right_answer)
    print("\n")
    
    print("Watson Assistent\n")
    print("predict_intent: %s"%watson_result[0])
    print("score: %.2f"%watson_result[1])
    print("right_intent: %s"%row.intent)
    if watson_result[0] == row.intent:
        watson_right_answer += 1
        print("is it right?: yes")
    else:
         print("is it right?: no")
    print("quantity of right answers: %s" %watson_right_answer)
    print("\n")

print("-------------------------------------------\n")
print("Total of questions: %d\n"%df_test.shape[0])
print("Omni NLP API\n") 
print("Total of right answers: %d"%omni_right_answer)
omni_acc = round(omni_right_answer/df_test.shape[0], 2)*100
print("Accuracy: %s\n"%omni_acc)

print("Watson Assistent\n") 
print("Total of right answers: %d"%watson_right_answer)
watson_acc = round(watson_right_answer/df_test.shape[0], 2)*100
print("Accuracy: %s"%watson_acc)

-------------------------------------------
sentence: quais os assuntos você trata


Omni NLP API

predict_intent: Bem_Estar
score: 0.19
right_intent: Bot_Capabilities
is it right?: no
quantity of right answers: 0


Watson Assistent

predict_intent: General_Positive_Feedback
score: 0.43
right_intent: Bot_Capabilities
is it right?: no
quantity of right answers: 0


-------------------------------------------
sentence: preciso trocar uma peça do meu carro, como devo proceder?


Omni NLP API

predict_intent: Veiculo_Problema
score: 0.22
right_intent: Veiculo_Problema
is it right?: yes
quantity of right answers: 1


Watson Assistent

predict_intent: Veiculo_Problema
score: 0.46
right_intent: Veiculo_Problema
is it right?: yes
quantity of right answers: 1


-------------------------------------------
sentence: Como modificar a senha do cartão Alelo VR, VA?


Omni NLP API

predict_intent: Esquecer_Senha_Vale_RefeicaoAlimentacao
score: 0.81
right_intent: Esquecer_Senha_Vale_RefeicaoAlimentaca

-------------------------------------------
sentence: Qual a fundação do CoE?


Omni NLP API

predict_intent: Fundacao_Valores
score: 0.37
right_intent: Fundacao_Valores
is it right?: yes
quantity of right answers: 13


Watson Assistent

predict_intent: Foco_Cliente
score: 0.49
right_intent: Fundacao_Valores
is it right?: no
quantity of right answers: 11


-------------------------------------------
sentence: Adotei recentemente meu filho e preciso colocar ele no meu plano odontologico


Omni NLP API

predict_intent: Incluir_Colaborador_Convenio
score: 0.67
right_intent: Incluir_Dependente_Plano_Saude
is it right?: no
quantity of right answers: 13


Watson Assistent

predict_intent: Ajuste_Informacao_Convenio
score: 0.37
right_intent: Incluir_Dependente_Plano_Saude
is it right?: no
quantity of right answers: 11


-------------------------------------------
sentence: Não estou cadastrada, e minha matricula aparece inválida, o que fazer?


Omni NLP API

predict_intent: Usuario_Invalido_N

-------------------------------------------
sentence: é possível tirar apenas @sys-number de férias e vender os demais dias?


Omni NLP API

predict_intent: Divisao_Ferias
score: 0.51
right_intent: Divisao_Ferias
is it right?: yes
quantity of right answers: 27


Watson Assistent

predict_intent: Agendamento_Alteracao_Ferias
score: 0.62
right_intent: Divisao_Ferias
is it right?: no
quantity of right answers: 21


-------------------------------------------
sentence: Gostaria de repassar os colaboradores que estão na minha gestão no PontoMais para outro gestor, por um periodo.


Omni NLP API

predict_intent: IncluirRemover_Colaborador
score: 0.34
right_intent: Repassar_Gestao_Colaborador
is it right?: no
quantity of right answers: 27


Watson Assistent

predict_intent: Repassar_Gestao_Colaborador
score: 0.65
right_intent: Repassar_Gestao_Colaborador
is it right?: yes
quantity of right answers: 22


-------------------------------------------
sentence: Quais são suas habilidades


Omni NL

-------------------------------------------
sentence: Colaborador necessita do número da Unimed do seu dependente, o mesmo não conseguiu pelo app da Unimed e 0800 do COE.


Omni NLP API

predict_intent: Verificar_Numero_Carteirinha_Convenio
score: 0.31
right_intent: Verificar_Numero_Carteirinha_Convenio
is it right?: yes
quantity of right answers: 38


Watson Assistent

predict_intent: General_Negative_Feedback
score: 0.56
right_intent: Verificar_Numero_Carteirinha_Convenio
is it right?: no
quantity of right answers: 31


-------------------------------------------
sentence: você está bem?


Omni NLP API

predict_intent: Bem_Estar
score: 0.88
right_intent: Bem_Estar
is it right?: yes
quantity of right answers: 39


Watson Assistent

predict_intent: Bem_Estar
score: 0.77
right_intent: Bem_Estar
is it right?: yes
quantity of right answers: 32


-------------------------------------------
sentence: Preciso do reembolso do meu relatório de despesas que está no Concur no valor de R$ 1500,00

-------------------------------------------
sentence: O que fazer caso meu usuário não esteja cadastrado ou apresente inválido?


Omni NLP API

predict_intent: Usuario_Invalido_NaoCadastrado
score: 0.14
right_intent: Usuario_Invalido_NaoCadastrado
is it right?: yes
quantity of right answers: 49


Watson Assistent

predict_intent: Incluir_Dependente_Plano_Saude
score: 0.33
right_intent: Usuario_Invalido_NaoCadastrado
is it right?: no
quantity of right answers: 39


-------------------------------------------
sentence: Como faço para prestar contas da devolução de um adiantamento no Concur?


Omni NLP API

predict_intent: Despesa_Pessoal_Marcado_Relatorio_Concur
score: 0.69
right_intent: Despesa_Pessoal_Marcado_Relatorio_Concur
is it right?: yes
quantity of right answers: 50


Watson Assistent

predict_intent: Procedimentos_Concur
score: 0.87
right_intent: Despesa_Pessoal_Marcado_Relatorio_Concur
is it right?: no
quantity of right answers: 39


-------------------------------------------

...

-------------------------------------------
sentence: Estou com a prestação de contas do meu cartão de credito corporativo relativo ao ano de 2019 em aberto. Não consegui fazer a prestação de nenhum mês. Preciso de ajuda, quem pode me ajuda?


Omni NLP API

predict_intent: Cartao_Clonado_Cambial
score: 0.11
right_intent: Procedimentos_Concur
is it right?: no
quantity of right answers: 99


Watson Assistent

predict_intent: Cartao_Clonado_Cambial
score: 0.39
right_intent: Procedimentos_Concur
is it right?: no
quantity of right answers: 86


-------------------------------------------
sentence: Só consigo ver numero cartao Dental.. é o mesmo numero do de saude? caso nao, como consigo o de saude - ja tentei no aplicativo e nao localizo.


Omni NLP API

predict_intent: Verificar_Numero_Carteirinha_Convenio
score: 0.11
right_intent: Verificar_Numero_Carteirinha_Convenio
is it right?: yes
quantity of right answers: 100


Watson Assistent

predict_intent: Download_Aplicativo_Convenio
score: 0

-------------------------------------------
sentence: Como solicito um ajuste de ponto pós fechamento ou reembolso?


Omni NLP API

predict_intent: Solicitar_Ponto_Pos_FechamentoReembolso
score: 0.43
right_intent: Solicitar_Ponto_Pos_FechamentoReembolso
is it right?: yes
quantity of right answers: 113


Watson Assistent

predict_intent: Procedimentos_Concur
score: 0.56
right_intent: Solicitar_Ponto_Pos_FechamentoReembolso
is it right?: no
quantity of right answers: 99


-------------------------------------------
sentence: O colaborador nao esta aparecendo na gestao do ponto do gestor.


Omni NLP API

predict_intent: IncluirRemover_Colaborador
score: 0.58
right_intent: Ponto_Colaborar_Não_Aparece
is it right?: no
quantity of right answers: 113


Watson Assistent

predict_intent: IncluirRemover_Colaborador
score: 0.68
right_intent: Ponto_Colaborar_Não_Aparece
is it right?: no
quantity of right answers: 99


-------------------------------------------
sentence: Recebi um adiantamento em 

-------------------------------------------
sentence: Como realizar a devolução do veículo?


Omni NLP API

predict_intent: Devolucao_Veiculo
score: 0.94
right_intent: Devolucao_Veiculo
is it right?: yes
quantity of right answers: 123


Watson Assistent

predict_intent: Devolucao_Veiculo
score: 0.92
right_intent: Devolucao_Veiculo
is it right?: yes
quantity of right answers: 108


-------------------------------------------
sentence: Me envolvi em um acidente com o meu carro


Omni NLP API

predict_intent: Veiculo_Provisorio
score: 0.08
right_intent: Veiculo_Problema
is it right?: no
quantity of right answers: 123


Watson Assistent

predict_intent: Veiculo_Problema
score: 0.38
right_intent: Veiculo_Problema
is it right?: yes
quantity of right answers: 109


-------------------------------------------
sentence: Ficarei ausente em virtude de viagem, como será feita gestão do ponto no período?


Omni NLP API

predict_intent: Solicitar_Ponto_Pos_FechamentoReembolso
score: 0.13
right_inten

-------------------------------------------
sentence: Como realizar um ajuste de ponto em caso de esquecimento?


Omni NLP API

predict_intent: Ajustar_Ponto
score: 0.93
right_intent: Ajustar_Ponto
is it right?: yes
quantity of right answers: 130


Watson Assistent

predict_intent: Ajustar_Ponto
score: 0.93
right_intent: Ajustar_Ponto
is it right?: yes
quantity of right answers: 117


-------------------------------------------
sentence: Como adicionar dependentes ao plano de saúde unimed?


Omni NLP API

predict_intent: Incluir_Dependente_Plano_Saude
score: 0.60
right_intent: Incluir_Dependente_Plano_Saude
is it right?: yes
quantity of right answers: 131


Watson Assistent

predict_intent: Incluir_Dependente_Plano_Saude
score: 0.60
right_intent: Incluir_Dependente_Plano_Saude
is it right?: yes
quantity of right answers: 118


-------------------------------------------
sentence: Quem realiza meus ajustes no ponto?


Omni NLP API

predict_intent: Ajustar_Ponto
score: 0.98
right_intent:

-------------------------------------------
sentence: Gostaria de alterar minhas férias para o fim do ano, como procedo?


Omni NLP API

predict_intent: Agendamento_Alteracao_Ferias
score: 0.44
right_intent: Agendamento_Alteracao_Ferias
is it right?: yes
quantity of right answers: 145


Watson Assistent

predict_intent: Agendamento_Alteracao_Ferias
score: 0.85
right_intent: Agendamento_Alteracao_Ferias
is it right?: yes
quantity of right answers: 131


-------------------------------------------
sentence: Preciso encaminhar o meu veiculo para outro colaborador


Omni NLP API

predict_intent: Transferencia_Veiculo
score: 0.24
right_intent: Transferencia_Veiculo
is it right?: yes
quantity of right answers: 146


Watson Assistent

predict_intent: Transferencia_Veiculo
score: 0.65
right_intent: Transferencia_Veiculo
is it right?: yes
quantity of right answers: 132


-------------------------------------------
sentence: FIZ UMA SOLICITAÇÃO PELO CONCUR DE REEMBOLSO E FOI APROVADO PELO GESTOR